# Heterogeneous robots achieve homeostasis in a dynamic environment through continuous adaptation and interaction

Include relevant libraries

In [ ]:
using Plots
using Statistics

Setup experimental and I/O constants

In [ ]:
DATA_PATH =             "../th"
VIDEO_FILE =            "video.mp4"

TARGET_REPLICAS =       100
EPOCHS_COUNT =          20_000
EPOCH_DURATION =        500
ROBOTS_COUNT =          50;

Count the number of completed replicas and epochs

In [ ]:
STEPS_COUNT = round(Int32, filesize("$DATA_PATH/clusters.dat") / 4 / ROBOTS_COUNT / EPOCH_DURATION)

REPLICAS_COUNT = max(floor(Int32, STEPS_COUNT / EPOCHS_COUNT), 1)
COMPLETED_EPOCHS = min(STEPS_COUNT, EPOCHS_COUNT)

"Run replicas: $REPLICAS_COUNT, Completed epochs: $COMPLETED_EPOCHS"

Calculate the seeds of the completed runs

In [ ]:
seeds = 1:ROBOTS_COUNT * REPLICAS_COUNT
seeds = seeds .|> e -> 100 * floor(Int32, (e - 1) / REPLICAS_COUNT + 1) + (e - 1) % REPLICAS_COUNT;

## Performance based measures (unordered data!)

Create a vector where to store all the performance of the robots

In [ ]:
P = Matrix{Float32}(undef, COMPLETED_EPOCHS, 0);

Load the data into the matrix

In [ ]:
data = Vector{Float32}(undef, COMPLETED_EPOCHS)

for path in map(s -> "$DATA_PATH/performance_r$s.dat", seeds)
    file = open("$DATA_PATH/$path", "r", lock=false)

    read!(file, data)
    P = [P data]

    close(file)
end

### Plot the average robot performance across the epochs

In [ ]:
plot(
    mean(P, dims=2)[1:COMPLETED_EPOCHS],
    xlabel="Epochs", ylabel="Performance [AVG]",
    legend=false
)
plot!([EPOCHS_COUNT / 2], linestyle=:dash, seriestype=:vline, xlims=(0, COMPLETED_EPOCHS))

## Cluster based measures

In [ ]:
file = open("$DATA_PATH/clusters.dat", "r", lock=false)

C = Matrix{Int32}(undef, REPLICAS_COUNT * COMPLETED_EPOCHS, EPOCH_DURATION)
s = Vector{Int32}(undef, ROBOTS_COUNT)

for i in 1:REPLICAS_COUNT * COMPLETED_EPOCHS
    for j in 1:EPOCH_DURATION
        read!(file, s)

        C[i, j] = length(Set(s))    
    end
end

close(file)

### Clusters count

In [ ]:
plot(
    mean(C, dims=2)[1:COMPLETED_EPOCHS],
    xlabel="Epochs", ylabel="Clusters count [AVG]",
    legend=false
)
plot!([EPOCHS_COUNT / 2], linestyle=:dash, seriestype=:vline, xlims=(0, COMPLETED_EPOCHS))